## **Предобработка данных. Категориальные признаки. Работа с текстами.**

Содержание семинара опирается на семинары, проводимые на ФКН. Во второй части работы отчасти использован туториал Kaggle по Word2Vec.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

В первой части семинара будем использовать немного видоизменённый для наших целей датасет https://archive.ics.uci.edu/ml/datasets/AutoUniv.

В нём присутствуют целочисленные, вещественнозначные и категориальные признаки.

In [ ]:
df = pd.read_csv('table.csv')
df.head()

In [ ]:
y = df['class']
X = df.drop(['class'], axis=1)
X.head()

In [ ]:
label_enc = preprocessing.LabelEncoder()
y = label_enc.fit_transform(y)
y

**Преобразование категориальных признаков**

Рассмотрим несколько типов предобработки данных. Она важна для корректной работы различных методов машинного обучения, в особенности, для линейных. 

Если в наших данных присутствуют категориальные признаки, то есть текстовые или порядковые признаки - те признаки, к которым нельзя относиться как к числам, то для дальнешей работы с такими данными надо преобразовать категориальные признаки в числа. 

Нельзя упускать из вида порядковые признаки. Например, если наши данные содержат в качестве столбца индекс местности, то алгоритм будет считать, что индекс 119331 > 119101, что для нас смысла не имеет. Мы хотим, чтобы различные индексы служили индикаторами различных географических областей. Для этого порядковые признаки также надо предобрабатывать и переводить в числовые.

Посмотрим, какие стоблцы нашей таблицы содержат категориальные признаки.

In [ ]:
cat_features_mask = (X.dtypes == "object").values
cat_features_mask

In [ ]:
len(cat_features_mask[cat_features_mask==True])

**1 способ кодирования: счётчики**

Мы можем посчитать, сколько раз каждое значение встречалось в таблице и заменить каждый категориальный признак соответствующим счетчиком.

In [ ]:
X_copy = X.copy()

In [ ]:
# your code here

**2 способ кодирования: OneHot-кодирование**

Пусть некоторый признак принимает значения из множества K. OneHotEncoder вместо одного признака создает K бинарных признаков - по одному на каждое возможное значение исходного признака.

In [ ]:
enc = preprocessing.OneHotEncoder(sparse=False, drop='first')
X_cat = enc.fit_transform(X_copy[X_copy.columns[cat_features_mask]])
X_cat = pd.DataFrame(data=X_cat)

In [ ]:
print(X_cat.shape)
X_cat.head()

**3 способ кодирования: хэширование**

HashingVectorizer преобразовывает строку в числовой массив заданной длиной с помощью хэш-функции. В этом методе в качестве входных параметров мы задаем желаемое количество новых признаков, а также токенизатор - обработчик текста (в нём мы можем сделать любую удобную нам предобработку текста: удалить редкие слова, удалить знаки препинания, оставить только слова из определенного списка и т.д.). Токенизатор возвращает текст, разбитый на токены, т.е. на слова.

In [ ]:
def my_tokenizer(s):
    return [elem for elem in s.split()]

Наиболее интересный для нас с точки зрения хэширования - столбец att9.

In [ ]:
X_copy['att9'].drop_duplicates()

Для применения HashingVectorize выбираем из столбца все различные значения(слова) без повторений, обучаем HashingVectorizer на этих словах и применяем ко всему столбцу. В итоге мы получаем разреженную матрицу. С ней умеют работать многие алгоритмы машинного обучения, но при желании можем перевести ее в numpy array.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

coder = HashingVectorizer(tokenizer=my_tokenizer, n_features=3, norm='l2', alternate_sign=True)

train_wo_duple = X_copy['att9'].drop_duplicates()
coder.fit(train_wo_duple)

coder.transform(X_copy['att9'].values).toarray()

**Заполнение пропусков**

В исходных данных могут быть пропущенные значения. Большинство методов машинного обучения не умеют с ними работать. Для этого мы должны каким-нибудь образом заполнить эти пропуски.

Способы заполнения пропусков:

* средним значением 
* медианой
* самым часто встречающимся значением
* каким-то одним новым значением (иногда пропуск в данных можно воспринимать как еще одно категориальное значение).
* можно взять часть данных для обучения, а на другой части данных предсказать пропущенные значения (тогда для решения основной задачи нельзя обучаться на первой части данных)

Посмотрим, есть ли пропуски в наших данных и много ли их.

In [ ]:
#индексы строк с NAN

X_real = X_copy[X_copy.columns[~cat_features_mask]]

print(np.any(np.isnan(X_real)))

np.array(pd.isnull(X_real).any(1)).nonzero()[0]

Заполним пропуски в данных с помощью встроенного метода Imputer, используя среднее значение признака.

In [ ]:
from sklearn.impute import SimpleImputer

mis_replacer = SimpleImputer(strategy="mean")
X_no_mis = pd.DataFrame(data=mis_replacer.fit_transform(X_real))

In [ ]:
np.any(np.isnan(X_no_mis))

**Масштабирование признаков**

Для применения в особенности линейных методов машинного обучения масштабирование признаков очень важно. Может быть так, что метод даст совершенно неправильный результат без масштабирования.

Масштабирование признаков можно выполнить, например, одним из следующих способов:
 - $x_{new} = \dfrac{x - \mu}{\sigma}$, где $\mu, \sigma$ — среднее и стандартное отклонение значения признака по всей выборке (StandardScaler в sklearn)
 - $x_{new} = \dfrac{x - x_{min}}{x_{max} - x_{min}}$, где $[x_{min}, x_{max}]$ — минимальный интервал значений признака (MinMaxScaler в sklearn)

In [ ]:
normalizer = preprocessing.StandardScaler()
X_standard_scaled = normalizer.fit_transform(X_no_mis)
X_standard_scaled_pd = pd.DataFrame(data=X_standard_scaled)
X_standard_scaled_pd.head()

In [ ]:
mm_scaler = preprocessing.MinMaxScaler()
X_mm_scaled = mm_scaler.fit_transform(X_no_mis)

Разберем пример данных, когда масштабирование признаков сильно влияет на результат работы алгоритмов машинного обучения. Рассмотрим точки, равномерно нанесенные на плоскость (в данном датасете в качестве признаков x и y выступают координаты - долгота и широта точек, все точки территориально находятся в Москве).

In [ ]:
X = pd.read_csv('scaler_example.csv')
X.head()

Посмотрим, как выглядят точки из датасета. Точки относятся к двум классам.

In [ ]:
%pylab inline

Xtrain = X[['lat','lon']].values
Ytrain = X['class'].values

figure(figsize=(10,10))
scatter(Xtrain[Ytrain==1][:,0], Xtrain[Ytrain==1][:,1], color='green')
scatter(Xtrain[Ytrain==0][:,0], Xtrain[Ytrain==0][:,1], color='red')

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(Xtrain, Ytrain)

def plot_decision_line(Xtrain, Ytrain, clf, h):

    x_min, x_max = Xtrain[:,0].min() - 0.01, Xtrain[:,0].max() + 0.01
    y_min, y_max = Xtrain[:,1].min() - 0.01, Xtrain[:,1].max() + 0.01
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(10,10))
    plt.contourf(xx, yy, Z)

    Ytrain = np.array(Ytrain)
    plt.scatter(Xtrain[Ytrain==1][:,0], Xtrain[Ytrain==1][:,1], color='green')
    plt.scatter(Xtrain[Ytrain==0][:,0], Xtrain[Ytrain==0][:,1], color='red')

    a = clf.coef_[0][0]
    b = clf.coef_[0][1]
    c = clf.intercept_

    K = -a * 1. / b
    B = -c * 1. / b

    xx0 = np.linspace(x_min, x_max)
    yy0 = K * xx0 + B

    plt.scatter(xx0, yy0)
    plt.show()
    
plot_decision_line(Xtrain, Ytrain, clf, 0.001)

   Мы видим, что SVM абсолютно не справился с задачей. Попробуем теперь масштабировать данные.

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
TrainScaled = sc.fit_transform(Xtrain)

clf.fit(TrainScaled, Ytrain)

In [ ]:
plot_decision_line(TrainScaled, Ytrain, clf, 0.01)

## **Работа с текстовыми признаками**

In [ ]:
from tqdm import tqdm
import re

%pylab inline

Одним из направлений машинного обучения является работа с текстами и извлечение полезной информации из текстов. Чтобы алгоритмы машинного обучения могли работать с текстами, необходимо перевести тексты в наборы чисел. Для этого применяют различные алгоритмы векторизации текстов. 

Будем изучать датасет, содержащий отзывы о фильмах. Отзывы могут быть положительные, либо отрицательные. Наша конечная задача - научиться различать положительные и отрицательные отзывы.

Загрузим датасет и уберем из него плохие строки (в которых нет оценки фильму).

In [ ]:
import codecs

with codecs.open('imdb_labelled.txt', encoding='utf-8') as thefile:
    print(thefile.read())

In [ ]:
responses = []
X = []
y = []

bad = 0
with codecs.open('imdb_labelled.txt', encoding='utf-8') as thefile:
    for row in tqdm(thefile.readlines()):
        # your code here

In [ ]:
bad_responses = list(filter(lambda review: 'awful' in review, X))
print(bad_responses[1])

In [ ]:
len(bad_responses)

In [ ]:
len(X)

Первые этапы обработки текста:

* приведение к нижнему регистру

* удаление пунктуации

* удаление всех символов, кроме символов нашего алфавита (в данном случае, латинского)

Удалить пунктуацию можно при помощи регулярных выражений. [Прикольный сайт для создания регулярок](https://regex101.com/).

In [ ]:
print(re.sub(r'[^\w\s]', '', bad_responses[1].lower()).strip())

In [ ]:
Texts = [re.sub(r'[^\w\s]', '', elem.lower()).strip() for elem in X]
Texts[35]

Посмотрим на распределение ответов в наших данных. 

In [ ]:
hist(y)

**1 способ векторизации: счётчик (CountVectorizer)**

Каждому слову соответствует количество его вхождений в текст.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(encoding='utf8', min_df=5)
vectorizer.fit(Texts)

In [ ]:
vectorizer.transform(Texts[:1])

In [ ]:
print(vectorizer.transform(Texts[:1]).indptr)
print(vectorizer.transform(Texts[:1]).indices)
print(vectorizer.transform(Texts[:1]).data)

**2 способ векторизации: TF-IDF**

Ещё один способ работы с текстовыми данными — TF-IDF (Term Frequency–Inverse Document Frequency). Рассмотрим коллекцию текстов $D$. Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

1. Term Frequency – количество вхождений слова в отношении к общему числу слов в тексте: 
    $$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$ где $n_{td}$ — количество вхождений слова $t$ в текст $d$.
2. Inverse Document Frequency $$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$ где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$.

Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину: $$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D).$

Признаковым описанием одного объекта $d \in D$ будет вектор $\bigg(\text{tf-idf}(t,d, D)\bigg)_{t\in V}$, где $V$ – словарь всех слов, встречающихся в коллекции $D$.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(Texts)

In [ ]:
vectorizer.transform(Texts[:1])

In [ ]:
print(vectorizer.transform(Texts[:1]).indptr)
print(vectorizer.transform(Texts[:1]).indices)
print(vectorizer.transform(Texts[:1]).data)

Применим два рассмотренных метода векторизации к задаче классификации отзывов на два класса (положительные и отрицательные).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

In [ ]:
vectorizer = CountVectorizer(encoding='utf8', min_df=5)
vectorizer.fit(Texts)

X = vectorizer.transform(Texts)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
preds = lr.predict_proba(X_test)[:,1]
print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(y_test, preds), accuracy_score(y_test, (preds > 0.5).astype(int))))

In [ ]:
vectorizer = TfidfVectorizer(encoding='utf8', min_df=5)
vectorizer.fit(Texts)

X = vectorizer.transform(Texts)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
preds = lr.predict_proba(X_test)[:,1]
print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(y_test, preds), accuracy_score(y_test, (preds > 0.5).astype(int))))

**Важность признаков**

В задачах, связанных с обработкой текстов, признаки как правило хорошо интерпретируемы. Для визуального контроля качества работы алгоритма можно посмотреть на те слова, которые алгоритм посчитал наиболее важными для данной задачи.

In [ ]:
weights = zip(vectorizer.get_feature_names(), lr.coef_[0])
weights = sorted(weights, key=lambda i: i[1])
for i in range(1,20):
    print('%s, %.2f' % weights[-i])
    
print('...')
for i in reversed(range(1,20)):
    print('%s, %.2f' % weights[i])

**3 способ векторизации: Word2Vec**

Word2Vec - это алгоритм, который собирает статистику по совместному появлению слов в фразах, а затем с помощью нейронных сетей решает задачу снижения размерности и выдает на выходе компактные векторные представления слов, в максимальной степени отражающие отношения этих слов в обрабатываемых текстах.

Нахождение связей между контекстами слов основано на предположении, что слова, находящиеся в похожих контекстах, имеют тенденцию значить похожие вещи, т.е. быть семантически близкими.

Применим векторизацию с помощью word2vec для наших данных. Кроме того, удалим stop-слова, то есть слова, часто встречающиеся во всех английских текстах - это ещё один полезный метод обработки текстов.

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download()

In [ ]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

In [ ]:
def delete_stopwords(review, remove_stopwords=True):
    
    words = review.split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)

Переведем строки нашего датасета в токенизированный вид и удалим из них стоп-слова - в этом виде они пригодны для использования word2vec.

In [ ]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review,tokenizer,remove_stopwords=True ):

    raw_sentences = tokenizer.tokenize(review.strip())

    sentences = []
    for sentence in raw_sentences:
        if len(sentence) > 0:
            sentences.append(delete_stopwords(sentence,remove_stopwords))
    return sentences

sentences = []
Y = []
for i in range(len(Texts)):
    if len(set(Texts[i])) == 1:
        continue
    Y.append(y[i])
    sentences += review_to_sentences(Texts[i], tokenizer)

Применим word2vec к токенизированному корпусу.

In [ ]:
num_features = 512         
min_word_count = 3           
num_workers = 4       
context = 5                                                                          
downsampling = 1e-5   

from gensim.models import word2vec


model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=42)

model.init_sims(replace=True)

model_name = str(num_features) + "features_word2vec"
model.save(model_name)

Теперь каждое слово корпуса имеет векторное представление

In [ ]:
model.wv['good']

В качестве одного из способов векторизовать текст (в нашем случае отзыв на фильм), можно усреднить векторы слов, входящих в этот текст. Так и сделаем.

In [ ]:
index2word_set = set(model.wv.index2word)

def normalize(x):
    return x / np.sqrt(np.dot(x, x))

def make_featurevec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0.
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec += model[word]
    if nwords > 0:
        featureVec = normalize(featureVec)
        # featureVec = np.divide(featureVec, nwords)
    return featureVec

def get_avg_featurevecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = make_featurevec(review, model, num_features)
        counter += 1
    return reviewFeatureVecs

trainDataVecs = get_avg_featurevecs(sentences, model, num_features)

Наконец, обучим классификатор на полученных признаках.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trainDataVecs, y, test_size=0.2, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
preds = lr.predict_proba(X_test)[:,1]
print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(y_test, preds), accuracy_score(y_test, (preds > 0.5).astype(int))))